<a href="https://colab.research.google.com/github/jamunozlab/introductory_mechanics_fall_2023/blob/main/projects/Phys_2320_project_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 7

When I showed in lecture the meme about physicists thinking that everything is simple harmonic motion, I was not really exagerating. This is because any function, but particularly oscillatory functions, can be described as being the weighted sum of sine waves of different frequencies. The [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) can take in an oscillatory wave and return the weights of the sine waves whose addition reassemble the original wave.

.

In this last project, you will decompose the position as a function of time and the velocity as a function of time generated using our now famous molecular dynamics code, for two different potentials.

In [ ]:
# Make our lives easier
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import numpy as np

In [ ]:
# Initialization
position = 0.0 # position in meters
velocity = 1.0 # velocity in meters per second
acceleration = 0.0 # acceleration in meters per second^2
force = 0.0 # force in Newtons
time = 0.0 # time in seconds
spring_constant = 1.0 # force constant in kg/second^2
mass = 1.0 # mass in kg
time_step = 0.001 # time step in seconds
g = 9.8 # acceleration due to gravity m/s^2
mu = 0.000 # coefficient of kinetic friction

In [ ]:
# A quadratic model for potential, this is just Hooke's law
# THIS IS THE CODE CELL THAT WILL BE REPLACED TO COMPLETE EXERCISE 7.2

potential_series = pd.Series([(1/2)*spring_constant*(x/100)**2 for x in range(-125,125)], index=[x/100 for x in range(-125,125)]) # ΔX ranges from -1.25 m to + 1.25 m // Arbitrary approximation
potential_series.plot(xlabel="position (m)", ylabel="Spring potential energy (J)")  # U = (1/2)k(Δx)2

In [ ]:
# Take the derivative of a pandas series at a given position using central difference
# We need this function to obtain forces from the potential

def take_derivative_at(potential_series, position):
  diff = np.inf  #Infinity
  closest = 0
  for index, pos in enumerate(potential_series.index): #position at each index is enumerated
    if abs(position - pos) < diff: # this IF will always execute the first iteration
      diff = abs(position - pos)
      closest = index # Closest = 0
  rise = potential_series.iloc[closest+1] - potential_series.iloc[closest-1]  #  ΔU = - Uf - Ui
  run = potential_series.index[closest+1] - potential_series.index[closest-1] # Δx = xf - xi
  slope = rise/run # ΔU / Δx

  return slope

In [ ]:
# Now that we've calculated Spring Force by approximating the derrivative of potential with respect to position (dU/dx)
# We can use Kinematics and Newtons second law to approximate the values of our remaining variables.

# Creating variable arrays
time_list = [time]
position_list = [position]
velocity_list = [velocity]
acceleration_list = [acceleration]
force_list = [force]

# Starting Loop
i=1
while i < 200000:
    time = time + time_step
    position = position + velocity*time_step + (1/2)*(acceleration*time_step**2) #Xf = xi + v(Δt)
    velocity = velocity + acceleration*time_step #Vf = Vi + a(Δt)
    spring_force = -1*take_derivative_at(potential_series=potential_series, position=position)  #Fs = -kΔx

    if velocity < 0: # moving to the left
      friction_force = mu*mass*g #friction to the right
    else: # moving to the right
      friction_force = -mu*mass*g # friction to the left
                                                             # We set the coefficient of friction to Zero, so friction will not affect
    if abs(velocity) < 0.001: # if moving extremely slowly
      friction_force = 0 # no friction because almost not moving

    force = spring_force + friction_force # Fnet = Fs + Fk
    acceleration = force/mass

# After all calculations have been computed they are then stored into our variable arrays

    time_list.append(time)
    position_list.append(position)
    velocity_list.append(velocity)
    acceleration_list.append(acceleration)
    force_list.append(force)
    i = i + 1

In [ ]:
# The position oscillation is here
# You can adjust the time window by changing the values in xlim
position_series = pd.Series(position_list, index=time_list)
position_series.plot(ylabel="position (m)", xlabel="time (s)", xlim=(0, 20))

In [ ]:
# The velocity oscillation is here
# You can adjust the time window by changing the values in xlim
velocity_series = pd.Series(position_list, index=time_list)
velocity_series.plot(ylabel="velocity (m/s)", xlabel="time (s)", xlim=(0, 20))

In [ ]:
# Fourier transform and frequency analysis of the position wave
# The numpy function fft is the Fourier transform,
# it finds the weights of the frequencies of the sine waves that make up the original function
# fftfreq computes those frequencies based on the amount of data N and sampling rate T
# The sampling rate is, of course, the time step of the simulation
data = np.array(position_series.values)
n = len(data)
d = time_step
yf = fft(data)
freq = fftfreq(n, d)

# Plotting
plt.plot(freq, np.abs(yf))
plt.title('Position frequency spectrum')
plt.ylabel('Intensity')
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 1])
plt.show()

In [ ]:
# Fourier transform and frequency analysis of the velocity wave
# Everything else is the same as before
data = np.array(position_series.values)
n = len(data)
d = time_step
yf = fft(data)
freq = fftfreq(n, d)

# Plotting
plt.plot(freq, np.abs(yf))
plt.title('Velocity frequency spectrum')
plt.ylabel('Intensity')
plt.xlabel('Frequency (Hz)')
plt.xlim([0, 1])
plt.show()

## Problem 7.1

7.1.1. Using a ruler, or by adjusting the arguments in xlim in the cells above, determine the period $T_x$ of the position oscillation and compute its frequency $f_x$ (remember that $f=1/T$).

7.1.2 Using a ruler, or by adjusting the arguments in xlim in the cells above, determine the period $T_v$ of the velocity oscillation and compute its frequency $f_v$ (remember that $f=1/T$).

7.1.3 By visual inspection of the position frequency spectrum, determine the most common frequency of the position oscillation and compare it to the result you got in 7.1.1. Are they about equal? Explain why that might be.

7.1.4 By visual inspection of the velocity frequency spectrum, determine the most common frequency of the velocity oscillation and compare it to the result you got in 7.1.2. Are they about equal? Explain why that might be.

7.1.5 The position and the velocity oscillations are both sinusoidal waves, since we are simulating a simple harmonic oscillator. Nevertheless, they are shifted by $\pi$ radians. Why do they have the same Fourier spectrum?

In [ ]:
## 7.1.1. What are the period and frequency of the position oscillation?
##
## Period answer:
## Frequency answer:

In [ ]:
## 7.1.2. What are the period and frequency of the velocity oscillation?
##
## Period answer:
## Frequency answer:

In [ ]:
## 7.1.3. What is the most common frequency of the position oscillation as determined by the Fourier transform
## and how does this number compare with your answer in 7.1.1? Explain.
##
## Frequency answer:
## Comparison answer:
## Explanation answer:

In [ ]:
## 7.1.4. What is the most common frequency of the velocity oscillation as determined by the Fourier transform
## and how does this number compare with your answer in 7.1.2? Explain.
##
## Frequency answer:
## Comparison answer:
## Explanation answer:

In [ ]:
## 7.1.5 Why are the frequency spectra of the position and velocity oscillations the same?
##
## Answer:

## Problem 7.2

Copy the code cell below that produces a quartic plus quadratic potential (rather than just quadratic) and replace the potential in the third cell from the top that you used for the previous problem. Restart your session and run all cells again. New position and velocity oscillations, as well as frequency spectra will be generated. The following questions refer to the quartic oscillations.

7.2.1 Using a ruler, or by adjusting the arguments in xlim in the cells above, determine the period $T_x$ of the position oscillation and compute its frequency $f_x$ (remember that $f=1/T$).

7.2.2 By visual inspection of the position frequency spectrum, determine the most common frequency of the position oscillation and compare it to the result you got in 7.1.2. Are they about equal? Explain why that might be.

7.2.3 Contrary to the frequency spectrum of the pure quadratic potential, the frequency spectrum of the quartic plus quadratic potential shows a second, smaller bump at higher frequency. What do you think those frequencies represent?  

In [ ]:
potential_series = pd.Series([(1/2)*spring_constant*(x/100)**4 - (1/2)*spring_constant*(x/100)**2 for x in range(-150,150)], index=[x/100 for x in range(-150,150)])
potential_series.plot(xlabel="position (m)", ylabel="Spring potential energy (J)")

In [ ]:
## 7.2.1. What are the period and frequency of the position oscillation?
##
## Period answer:
## Frequency answer:

In [ ]:
## 7.2.2. What is the most common frequency of the position oscillation as determined by the Fourier transform
## and how does this number compare with your answer in 7.2.1? Explain.
##
## Period answer:
## Frequency answer:

In [ ]:
## 7.2.3. Why is there a smaller bump at higher frequency in the frequency spectrum?
##
## Period answer:
## Frequency answer:

## Problem 7.3

One of the most interest features of the universe is that, to a large degree, it can be described as an agregation of models that do not interact.

7.3.1.Evaluate the cell below and explain how this relates to the 'modified' simple harmonic motion produced by the quartic plus quadratic potential?

In [ ]:
Fs = 8000
sample = 80000

# This generates the low frequency harmonic wave
f = 0.2
x = np.arange(sample)
x = np.array(time_list)
y1 = np.sin(2 * np.pi * f * x)
plt.xlim(0,20)
plt.plot(x, y1)

# This generates the high frequency harmonic wave
f = 0.55
y2 = np.sin(2 * np.pi * f * x )
plt.plot(x, y2)
plt.show()

# Adding the 2 waves, multiplying one of them times 10 percent due to
# them being less common in the frequency spectrum approximately
# reproduces the triangular wave of the quadratic plus quartic potential
y3 = 0.1*y2+y1
plt.plot(x, y3)
plt.xlim(0,20)
plt.show()

In [ ]:
## 7.3.1 Why can you decompose an oscillation into a sum of harmonic oscillations?
##
## Answer: